In [1]:
library(tidyverse, quietly = TRUE)
library(psych, quietly = TRUE)
library(Hmisc, quietly = TRUE)
library(car)
library(compositions)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘psych’


The following objects are masked from ‘package:ggplot2’:

    %+%, alpha



Attaching package: ‘Hmisc’


The following object is masked from ‘package:psych’:

    describe


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:psych’:

In [2]:
data <- read.csv("../data/combined_data.csv")
head(data, 1)

,NAME,GEOID,with_internet,work_from_home,sixtyfive_and_older,median_income,number_providers,mean_lowest_cost,mean_d_mbps,mean_u_mbps,mean_lat_ms
,<chr>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,"Census Tract 9601, Asotin County, Washington",53003960100,1569,145,1337,73942,12,58,110.3274,41.7821,51.70445


In [3]:
data <- data %>% dplyr::select(-NAME, -GEOID)

head(data)

,with_internet,work_from_home,sixtyfive_and_older,median_income,number_providers,mean_lowest_cost,mean_d_mbps,mean_u_mbps,mean_lat_ms
,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1569,145,1337,73942,12,58.00000,110.3274,41.78210,51.70445
2,1780,421,1352,72988,13,60.07692,145.7848,22.88731,24.62630
3,1156,6,396,38077,11,60.00000,173.4079,27.06202,29.98058
4,805,29,647,38777,8,62.50000,127.6005,23.53829,22.16379
5,1198,26,635,42639,8,62.50000,192.4299,21.68565,21.66087
6,1392,115,909,53947,9,58.88889,151.0552,23.52123,26.18557


In [4]:
KMO(data)

Kaiser-Meyer-Olkin factor adequacy
Call: KMO(r = data)
Overall MSA =  0.63
MSA for each item = 
      with_internet      work_from_home sixtyfive_and_older       median_income 
               0.57                0.74                0.55                0.65 
   number_providers    mean_lowest_cost         mean_d_mbps         mean_u_mbps 
               0.71                0.69                0.65                0.60 
        mean_lat_ms 
               0.67 

In [5]:
scaled_data <- scale(data)

In [6]:
pca_data <- prcomp(scaled_data, scale = FALSE)

# disable scientific notation
options(scipen = 999)
eigenvectors <- -pca_data$rotation
eigenvalues <- pca_data$sdev^2

index_col <- as.data.frame(-pca_data$x) %>%
    dplyr::select(PC1) %>%
    rename(index = PC1)

index_data <- bind_cols(scaled_data, index_col)
head(index_data)

,with_internet,work_from_home,sixtyfive_and_older,median_income,number_providers,mean_lowest_cost,mean_d_mbps,mean_u_mbps,mean_lat_ms,index
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.5167430,0.1292741,2.23493253,0.07418486,1.7094218,-0.18458632,-1.05913969,0.2831385,1.17991288,-1.4973910
2,0.9194434,2.9303783,2.28240921,0.07416551,2.2108891,0.41923513,-0.64056563,-0.4304467,-0.15416598,-0.4445651
3,-0.2714809,-1.2814270,-0.74343819,0.07345717,1.2079545,0.39687137,-0.31447576,-0.2727836,0.10962741,-0.9229616
4,-0.9413759,-1.0480017,0.05100501,0.07347137,-0.2964475,1.12369349,-0.85523159,-0.4058615,-0.27548794,-0.4642621
5,-0.1913226,-1.0784484,0.01302366,0.07354973,-0.2964475,1.12369349,-0.08992031,-0.4758288,-0.30026584,0.1856894
6,0.1789328,-0.1751938,0.88026444,0.07377917,0.2050198,0.07383932,-0.57834859,-0.4065058,-0.07734433,-0.3719643


In [7]:
as.list(rcorr(as.matrix(index_data), type = "pearson")$P[, ncol(index_data)])

$with_internet
[1] 0.0000000000000184297

$work_from_home
[1] 0.04883481

$sixtyfive_and_older
[1] 0.1971623

$median_income
[1] 0.7867191

$number_providers
[1] 0

$mean_lowest_cost
[1] 0.00000002783109

$mean_d_mbps
[1] 0

$mean_u_mbps
[1] 0.00000000000009725554

$mean_lat_ms
[1] 0

$index
[1] NA

In [8]:
as.list(rcorr(as.matrix(index_data), type = "pearson"))$P

,with_internet,work_from_home,sixtyfive_and_older,median_income,number_providers,mean_lowest_cost,mean_d_mbps,mean_u_mbps,mean_lat_ms,index
with_internet,NA,0.0000000002997904,0.00000000000000,0.03006990,0.082203264746717686,0.06558715015739,0.000001719724394889,0.00690289845388036838,0.0012541075067722,0.00000000000001842970
work_from_home,0.0000000002997904147,NA,0.00000004534735,0.62244972,0.692796119793771537,0.93350819129816,0.672635578892659680,0.43009326946822890037,0.9085912323262457,0.04883481104472586765
sixtyfive_and_older,0.0000000000000000000,0.0000000453473543,NA,0.04477147,0.476702946467304045,0.49718826894047,0.569411922064836595,0.02203015631983151756,0.1693793288053422,0.19716228248808564949
median_income,0.0300698982044131924,0.6224497181702566,0.04477147246482,NA,0.767570871280282230,0.46966892178760,0.521089097196837159,0.80723851787517242329,0.4313383103711321,0.78671911871383670700
number_providers,0.0822032647467176858,0.6927961197937715,0.47670294646730,0.76757087,NA,0.00000526142273,0.000000000006029399,0.82692993872093989438,0.0000000002296336,0.00000000000000000000
mean_lowest_cost,0.0655871501573890292,0.9335081912981564,0.49718826894047,0.46966892,0.000005261422731273,NA,0.007630242115620689,0.25618495750335812389,0.0141555670114635,0.00000002783108787874
mean_d_mbps,0.0000017197243948885,0.6726355788926597,0.56941192206484,0.52108910,0.000000000006029399,0.00763024211562,NA,0.00000001366549318149,0.0000000000000000,0.00000000000000000000
mean_u_mbps,0.0069028984538803684,0.4300932694682289,0.02203015631983,0.80723852,0.826929938720939894,0.25618495750336,0.000000013665493181,NA,0.0000000029104892,0.00000000000009725554
mean_lat_ms,0.0012541075067722396,0.9085912323262457,0.16937932880534,0.43133831,0.000000000229633645,0.01415556701146,0.000000000000000000,0.00000000291048918299,NA,0.00000000000000000000
index,0.0000000000000184297,0.0488348110447259,0.19716228248809,0.78671912,0.000000000000000000,0.00000002783109,0.000000000000000000,0.00000000000009725554,0.0000000000000000,NA


In [9]:
data1.2 <- data %>% dplyr::select(-median_income, -sixtyfive_and_older)
scaled_data1.2 <- scale(data1.2)

pca_data1.2 <- prcomp(scaled_data1.2, scale = FALSE)
eigenvectors1.2 <- -pca_data1.2$rotation
eigenvalues1.2 <- pca_data1.2$sdev^2

index_col1.2 <- as.data.frame(-pca_data1.2$x) %>%
    dplyr::select(PC1) %>%
    rename(index = PC1)

index_data1.2 <- bind_cols(scaled_data1.2, index_col1.2)

In [10]:
as.list(rcorr(as.matrix(index_data1.2), type = "pearson")$P[, ncol(index_data1.2)])

$with_internet
[1] 0.00000000000190048

$work_from_home
[1] 0.12573

$number_providers
[1] 0

$mean_lowest_cost
[1] 0.00000003139127

$mean_d_mbps
[1] 0

$mean_u_mbps
[1] 0.00000000000001976197

$mean_lat_ms
[1] 0

$index
[1] NA

In [11]:
data1.3 <- data %>% dplyr::select(-median_income, -sixtyfive_and_older, -work_from_home)
scaled_data1.3 <- scale(data1.3)

pca_data1.3 <- prcomp(scaled_data1.3, scale = FALSE)
eigenvectors1.3 <- -pca_data1.3$rotation
eigenvalues1.3 <- pca_data1.3$sdev^2

index_col1.3 <- as.data.frame(-pca_data1.3$x) %>%
    dplyr::select(PC1) %>%
    rename(index = PC1)

index_data1.3 <- bind_cols(scaled_data1.3, index_col1.3)

In [12]:
library(leaps)
subset <- regsubsets(index ~ ., index_data1.3)
summary(subset)

Subset selection object
Call: regsubsets.formula(index ~ ., index_data1.3)
6 Variables  (and intercept)
                 Forced in Forced out
with_internet        FALSE      FALSE
number_providers     FALSE      FALSE
mean_lowest_cost     FALSE      FALSE
mean_d_mbps          FALSE      FALSE
mean_u_mbps          FALSE      FALSE
mean_lat_ms          FALSE      FALSE
1 subsets of each size up to 6
Selection Algorithm: exhaustive
         with_internet number_providers mean_lowest_cost mean_d_mbps
1  ( 1 ) " "           " "              " "              "*"        
2  ( 1 ) " "           " "              " "              "*"        
3  ( 1 ) " "           "*"              " "              "*"        
4  ( 1 ) "*"           "*"              " "              " "        
5  ( 1 ) "*"           "*"              " "              "*"        
6  ( 1 ) "*"           "*"              "*"              "*"        
         mean_u_mbps mean_lat_ms
1  ( 1 ) " "         " "        
2  ( 1 ) " "      

In [13]:
forward <- regsubsets(index ~ ., index_data1.3, method = "forward")

results <- summary(forward)

results

Subset selection object
Call: regsubsets.formula(index ~ ., index_data1.3, method = "forward")
6 Variables  (and intercept)
                 Forced in Forced out
with_internet        FALSE      FALSE
number_providers     FALSE      FALSE
mean_lowest_cost     FALSE      FALSE
mean_d_mbps          FALSE      FALSE
mean_u_mbps          FALSE      FALSE
mean_lat_ms          FALSE      FALSE
1 subsets of each size up to 6
Selection Algorithm: forward
         with_internet number_providers mean_lowest_cost mean_d_mbps
1  ( 1 ) " "           " "              " "              "*"        
2  ( 1 ) " "           " "              " "              "*"        
3  ( 1 ) " "           " "              "*"              "*"        
4  ( 1 ) "*"           " "              "*"              "*"        
5  ( 1 ) "*"           "*"              "*"              "*"        
6  ( 1 ) "*"           "*"              "*"              "*"        
         mean_u_mbps mean_lat_ms
1  ( 1 ) " "         " "        
2

In [14]:
model <- lm(
    index ~ mean_d_mbps, index_data1.3
)
summary(model)


Call:
lm(formula = index ~ mean_d_mbps, data = index_data1.3)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.70584 -0.43971 -0.02255  0.45061  1.68436 

Coefficients:
                         Estimate            Std. Error t value
(Intercept) 0.0000000000000001285 0.0511065819026084930     0.0
mean_d_mbps 1.4709618235742687453 0.0512467922673198531    28.7
                       Pr(>|t|)    
(Intercept)                   1    
mean_d_mbps <0.0000000000000002 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.6914 on 181 degrees of freedom
Multiple R-squared:  0.8199,	Adjusted R-squared:  0.8189 
F-statistic: 823.9 on 1 and 181 DF,  p-value: < 0.00000000000000022


In [15]:
model2 <- lm(
    index ~ mean_lat_ms + mean_d_mbps, 
    index_data1.3
)
summary(model2)


Call:
lm(formula = index ~ mean_lat_ms + mean_d_mbps, data = index_data1.3)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.19923 -0.40893 -0.01228  0.38916  1.35442 

Coefficients:
                             Estimate                Std. Error t value
(Intercept) -0.0000000000000000008643  0.0434782552565973340331   0.000
mean_lat_ms -0.6571177518744153456254  0.0784928270836868224869  -8.372
mean_d_mbps  0.9245283099032233220527  0.0784928270836868224869  11.779
                        Pr(>|t|)    
(Intercept)                    1    
mean_lat_ms   0.0000000000000155 ***
mean_d_mbps < 0.0000000000000002 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.5882 on 180 degrees of freedom
Multiple R-squared:  0.8704,	Adjusted R-squared:  0.8689 
F-statistic: 604.2 on 2 and 180 DF,  p-value: < 0.00000000000000022


In [16]:
model3 <- lm(
    index ~ mean_lat_ms + mean_d_mbps + mean_lowest_cost,
    index_data1.3
)
summary(model3)


Call:
lm(formula = index ~ mean_lat_ms + mean_d_mbps + mean_lowest_cost, 
    data = index_data1.3)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.51984 -0.35446 -0.00827  0.31214  1.38770 

Coefficients:
                               Estimate             Std. Error t value
(Intercept)      -0.0000000000000001416  0.0348208194465178464    0.00
mean_lat_ms      -0.6366596523024188325  0.0628959988089049377  -10.12
mean_d_mbps       0.8709064547483797991  0.0630878721772683698   13.80
mean_lowest_cost  0.3591995464179494890  0.0356302111979138053   10.08
                            Pr(>|t|)    
(Intercept)                        1    
mean_lat_ms      <0.0000000000000002 ***
mean_d_mbps      <0.0000000000000002 ***
mean_lowest_cost <0.0000000000000002 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.471 on 179 degrees of freedom
Multiple R-squared:  0.9173,	Adjusted R-squared:  0.9159 
F-statistic: 661.9 on 3 and 179 DF,  

In [17]:
model4 <- lm(
    index ~ mean_lat_ms + 
    mean_d_mbps + 
    mean_lowest_cost +
    with_internet,
    index_data1.3
)
summary(model4)


Call:
lm(formula = index ~ mean_lat_ms + mean_d_mbps + mean_lowest_cost + 
    with_internet, data = index_data1.3)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.47743 -0.24823 -0.01983  0.24207  1.20332 

Coefficients:
                              Estimate            Std. Error t value
(Intercept)      -0.000000000000000162  0.028348155644931029   0.000
mean_lat_ms      -0.685585589272718732  0.051457824634856708 -13.323
mean_d_mbps       0.733445105870662029  0.053321242265313255  13.755
mean_lowest_cost  0.337422230695737679  0.029095743642552484  11.597
with_internet     0.292861755568714688  0.030520819627181200   9.595
                            Pr(>|t|)    
(Intercept)                        1    
mean_lat_ms      <0.0000000000000002 ***
mean_d_mbps      <0.0000000000000002 ***
mean_lowest_cost <0.0000000000000002 ***
with_internet    <0.0000000000000002 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3835 on 1

In [18]:
model5 <- lm(
    index ~ mean_lat_ms +
        mean_d_mbps +
        mean_lowest_cost +
        with_internet +
        number_providers,
    index_data1.3
)
summary(model5)


Call:
lm(formula = index ~ mean_lat_ms + mean_d_mbps + mean_lowest_cost + 
    with_internet + number_providers, data = index_data1.3)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.36749 -0.19468 -0.09555  0.12350  1.12830 

Coefficients:
                                Estimate              Std. Error t value
(Intercept)      -0.00000000000000002238  0.02136007448136309494    0.00
mean_lat_ms      -0.64644137967919168553  0.03891746614605681720  -16.61
mean_d_mbps       0.63162359424861880264  0.04111130247879486027   15.36
mean_lowest_cost  0.26404730632427958481  0.02280506922083241211   11.58
with_internet     0.30883167272258182923  0.02303773797240890944   13.40
number_providers -0.29839189613928263078  0.02553819579576480439  -11.68
                            Pr(>|t|)    
(Intercept)                        1    
mean_lat_ms      <0.0000000000000002 ***
mean_d_mbps      <0.0000000000000002 ***
mean_lowest_cost <0.0000000000000002 ***
with_internet    <0.000000000

In [19]:
model5


Call:
lm(formula = index ~ mean_lat_ms + mean_d_mbps + mean_lowest_cost + 
    with_internet + number_providers, data = index_data1.3)

Coefficients:
            (Intercept)              mean_lat_ms              mean_d_mbps  
-0.00000000000000002238  -0.64644137967919168553   0.63162359424861880264  
       mean_lowest_cost            with_internet         number_providers  
 0.26404730632427958481   0.30883167272258182923  -0.29839189613928263078  


In [20]:
model6 <- lm(
    index ~ mean_lat_ms +
        mean_d_mbps +
        mean_lowest_cost +
        with_internet +
        number_providers +
        mean_u_mbps,
    index_data1.3
)
summary(model6)

Warning message in summary.lm(model6):
“essentially perfect fit: summary may be unreliable”



Call:
lm(formula = index ~ mean_lat_ms + mean_d_mbps + mean_lowest_cost + 
    with_internet + number_providers + mean_u_mbps, data = index_data1.3)

Residuals:
                    Min                      1Q                  Median 
-0.00000000000000151116 -0.00000000000000012422  0.00000000000000000457 
                     3Q                     Max 
 0.00000000000000013981  0.00000000000000163229 

Coefficients:
                                Estimate              Std. Error
(Intercept)       0.00000000000000009084  0.00000000000000001991
mean_lat_ms      -0.54030991517724169526  0.00000000000000003703
mean_d_mbps       0.55737738214154430860  0.00000000000000003867
mean_lowest_cost  0.24468806134217044312  0.00000000000000002130
with_internet     0.28580825950903210320  0.00000000000000002153
number_providers -0.38548041995769960488  0.00000000000000002457
mean_u_mbps       0.32747807056422123528  0.00000000000000002294
                                t value             Pr(>|t|

In [21]:
as.list(vif(model5))

$mean_lat_ms
[1] 3.301444

$mean_d_mbps
[1] 3.68415

$mean_lowest_cost
[1] 1.133646

$with_internet
[1] 1.156896

$number_providers
[1] 1.421658